In [14]:
%load_ext autoreload
%autoreload 2
from tables import *
tables = Tables()
dic = tables.interventions()
dic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/joachimmuth/Documents/EPFL/ADA/parliament-viz/scraping/tables.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  transcripts['PersonNumber'] = transcripts['PersonNumber'].astype(int)
/Users/joachimmuth/Documents/EPFL/ADA/parliament-viz/scraping/tables.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

[INFO] JSON created in file  data/interventions.json


{'2310': [{'int': 8, 'median': 17, 'year': 2007},
  {'int': 229, 'median': 18, 'year': 2016},
  {'int': 70, 'median': 22, 'year': 2013},
  {'int': 66, 'median': 25, 'year': 2009},
  {'int': 4, 'median': 21, 'year': 2006},
  {'int': 142, 'median': 21, 'year': 2015},
  {'int': 75, 'median': 23, 'year': 2014},
  {'int': 12, 'median': 19, 'year': 2008},
  {'int': 6, 'median': 18, 'year': 2003},
  {'int': 22, 'median': 22, 'year': 2004},
  {'int': 2, 'median': 19, 'year': 2001},
  {'int': 6, 'median': 22, 'year': 2005},
  {'int': 91, 'median': 22, 'year': 2010},
  {'int': 7, 'median': 21, 'year': 2002},
  {'int': 68, 'median': 22, 'year': 2012},
  {'int': 141, 'median': 19, 'year': 2011}],
 '2450': [{'int': 0, 'median': 18, 'year': 2016}],
 '2268': [{'int': 4, 'median': 21, 'year': 2002},
  {'int': 10, 'median': 18, 'year': 2003},
  {'int': 15, 'median': 15, 'year': 2000},
  {'int': 9, 'median': 19, 'year': 2001}],
 '1073': [{'int': 0, 'median': 18, 'year': 2016}],
 '3071': [{'int': 3, 'med

In [15]:
dic['3079']

[{'int': 0, 'median': 18, 'year': 2016}]

In [16]:
dic['3078']

[{'int': 8, 'median': 18, 'year': 2016}]

In [18]:
dic['2713']

[{'int': 8, 'median': 19, 'year': 2008},
 {'int': 5, 'median': 22, 'year': 2010},
 {'int': 3, 'median': 19, 'year': 2011},
 {'int': 7, 'median': 23, 'year': 2014},
 {'int': 13, 'median': 25, 'year': 2009},
 {'int': 23, 'median': 18, 'year': 2016},
 {'int': 15, 'median': 22, 'year': 2012},
 {'int': 11, 'median': 22, 'year': 2013},
 {'int': 18, 'median': 21, 'year': 2015}]

In [ ]:
person = pd.read_csv('data/Person.csv')
person.head()

In [ ]:
person.dropna(axis=0, subset=['PersonIdCode'], inplace=True)

In [ ]:

person.PersonIdCode = person.PersonIdCode.astype(int)
person.PersonIdCode.head()

In [ ]:


def interventions(filename=None):
#     transcripts = self.df['Transcript']
#     sessions = self.df['Session']
#     persons = self.df['Person']

    transcripts = pd.read_csv('data/Transcript.csv')
    sessions = pd.read_csv('data/Session.csv')
    persons = pd.read_csv('data/Person.csv')

    if filename is None:
        filename = 'interventions'

    def word_counter(df):
        if type(df['Text']) == float:
            return 0
        else:
            return len(df['Text'].split())

    def define_year(df):
        return year_dict[int(df['IdSession'])]

    def get_year(df):
        return int(df.StartDate[:4])

    # filter transcription with PersonIdField
    transcripts = transcripts[np.isfinite(transcripts['PersonNumber'])]
    transcripts['PersonNumber'] = transcripts['PersonNumber'].astype(int)

    # filter long intervention
    transcripts['NumberWord'] = transcripts.apply(word_counter, axis=1)
    df_long = transcripts.loc[transcripts.NumberWord > 30]

    # link session number to year
    sessions.set_index('ID', inplace=True)
    sessions['StartYear'] = sessions.apply(get_year, axis=1)
    year_dict = sessions['StartYear'].to_dict()
    year_dict = defaultdict(lambda: 0, year_dict)

    # add year field
    df_long['year'] = df_long.apply(define_year, axis=1)

    # table : PersonNumber, year, interventions
    interventions = pd.DataFrame(df_long.groupby(['PersonNumber', 'year']).size().rename('Counts'))
    interventions = interventions.reset_index()

    # table : year, median intervention per person
    median = interventions.groupby('year').agg('median')
    median = median['Counts'].rename('median').astype(int)

    # table personNumber, PersonIdCode
    persons = persons.dropna(axis=0, subset=['PersonNumber', 'PersonIdCode'])
    persons.PersonIdCode = persons.PersonIdCode.astype(int)
    persons = persons.set_index('PersonNumber')
    persons = persons['PersonIdCode']

    # in main table, link to PersonIdCode and median per year
    interventions = interventions.join(persons, how='inner', on='PersonNumber')
    interventions = interventions.join(median, how='inner', on='year')

    # create the dictionnary for Json
    interventions.sort_values(by='PersonIdCode', inplace=True)
    persons_id = interventions.PersonIdCode.unique()

    dic = {}
    print(interventions)
    for person_id in persons_id:
        arr = list()
        for row in interventions.loc[interventions.PersonIdCode == person_id].values:
            print(row)
            internal_dic = {'year': int(row[1]), 'int': int(row[2]), 'median': int(row[4])}
            arr.append(internal_dic)
        dic[str(person_id)] = arr

    filepath = 'data/' + filename + '.json'
    with open(filepath, 'w') as fp:
        json.dump(dic, fp)
        print("[INFO] JSON created in file ", filepath)

    return interventions

interventions('coucou')

In [ ]:
sessions = pd.read_csv('data/Session.csv')

In [ ]:
sessions.set_index('ID', inplace=True)
sessions.head()
